# Data preprocessing: more art than science?

## Contents of this notebook:
<ol>
<li>Load and examine your data</li>
<li>Merging two dataframes</li>
<li>Removing features that you do not need</li>
<li>Making your data machine-readable</li>
<li>Handling not available (NA) and inf data</li>
<li>Removing columns with a standard deviation of 0</li>
<li>Feature scaling</li>
<li>Data visualization</li>
<li>Loading local files into Google Colab</li>
<li>Mini assignment: data visualization</li>
</ol>

## 1. Load and examine your data

### 1.1 Preliminary examination of your data

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

df = pd.read_csv("data_csv/unrestricted_HCP_behavioral.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'data_csv/unrestricted_HCP_behavioral.csv'

Quick quality check: is the dataframe shape what you expect it to be? You should know how many subjects (ie rows) and features (ie columns) you are expecting your dataframe (`df`) to have prior to any downstream analysis.

In [ ]:
print(f"df type: {type(df)}")
print(f"df shape: {df.shape}")

In [ ]:
df.head(4)

In [ ]:
df.tail(4)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df['Gender'].unique()

In [ ]:
df['Gender'].value_counts()

In [ ]:
df[df['Gender']=="F"] # Display only the rows of female subjects

### 1.2 Documentation
Woah! So many columns and abbreviations! What do they all mean? Make sure you know where your [dataset's documentation](https://www.humanconnectome.org/storage/app/media/documentation/s1200/HCP_S1200_Release_Reference_Manual.pdf) is.

https://wiki.humanconnectome.org/#resources-for-public-hcp-data-users

Unfortunately, thorough documentation is not always available. Some data types are also very field-specific and require the help of experts, which is part of what makes machine learning so wonderfully interdisciplinary.

In [ ]:
df.columns

Handpicking the columns you want to work with:

In [ ]:
basics = ['Subject','Gender','Age','PSQI_BedTime']
df[basics]

Let's also add all of some cognitive variables to the mix! Specifically, we'll select the measures related to fluid intelligence (they start with `PMAT` for Penn Matrix Test) and impulsivity (they start with `DDisc` for Delay Discounting)

In [ ]:
cognition = ['Subject','Gender','Age','PSQI_BedTime']
for col in df.columns:
    if (col.find("PMAT")!=-1 or col.find("DDisc")!=-1):
        cognition.append(col)
print(f"List of variables we will be looking at: {cognition}") # PS: f-strings will be very useful for you in your Python journey!

Okay! So let's actually select this subset of our data, and make it a separate dataframe!

In [ ]:
df_cognition = df[cognition]

In [ ]:
df_cognition.head()

In [ ]:
df_cognition.shape

### 2. Merging two dataframes

So for our dataset of 1206 subjects, we have information about their gender, age-range and a variety of cognitive measures. It would be cool to be able to integrate some other data as well. You have been provided with a separate file that contains brain structure volume data obtained from the neuroimaging data of these same subjects (note that not all subjects in the HCP have neuroimaging data)

In [ ]:
df_volumes = pd.read_csv("data_csv/HCP_volumes.csv")

In [ ]:
df_volumes

In [ ]:
df_volumes['TBV'].mean() # mean TBV!

In [ ]:
df_volumes['TBV'].std() # standard deviation of TBV!

List all our subjects whose TBV is above average! (PS: another way to write this: `df_volumes[df_volumes['TBV']>1406181]`)

In [ ]:
df_volumes[df_volumes['TBV']>df_volumes['TBV'].mean()]

Ok! Let's merge our dataframes. Do they have the same number of subjects?

In [ ]:
print(df_volumes.shape)
print(df_cognition.shape)

No! We only have volume data for 1086 subjects... So how do we create a new dataframe, where we only keep the subjects for which we have all of our features?

In [ ]:
df_final=pd.merge(df_cognition,df_volumes, on='Subject')
df_final.head()

In [ ]:
df_final.shape

Mission accomplished! We have 1086 and 14 + 22 features.

## 3. Removing features that you do not need

In [ ]:
df_final.columns

Do we have any columns that repeat themselves? Yes we do! `DDisc_AUC_200` and `DDisc_AUC_200.1` might be the same. Let's check

In [ ]:
df_final["DDisc_AUC_200"].equals(df_final["DDisc_AUC_200.1"])

In [ ]:
for col in df_final.columns:
  for col2 in df_final.columns:
    if(col != col2):
      if df_final[col].equals(df_final[col2])
      # now compare contents of 2 col
      print(col,col2)

Nice! Let's remove column `DDisc_AUC_200.1` then.

In [ ]:
df_final.drop('DDisc_AUC_200.1', inplace=True, axis=1)

In [ ]:
df_final.columns

## 4. Making your data machine-readable

To be machine-readable, your variables need to be numerical. Want to check?

In [ ]:
df_final.dtypes

We have 3 columns that are non-numerical: `Gender`,`Age`,`PSQI_BedTime`. Let's figure out how to handle them, one at a time. First of all, we know that the `Gender` column is categorical:

In [ ]:
df_final['Gender'].value_counts()

### 4.1 One-hot encoding or binarizing your data
In this specific dataset, the `Gender` column has one of two values: `M` or `F`. Given that we only have two categories here, you can just replace all of your `M` values with 1 and `F` values with 2.

Indeed, it has one of two values: `M` or `F`. Given that we only have two categories here, you can easily just set all of the `M`s to 1 and the `F` to 2:

In [ ]:
df_final['Gender'] = df_final['Gender'].replace('M',1)
df_final['Gender'] = df_final['Gender'].replace('F',2)
df_final

However, suppose that you actually had more than 2 numerical values for this feature (e.g. `M`,`F`,`other`). If you just convert categorical variables to numerical values (ex: `M`=1,`F`=2,`other`=3), you give a "distance" to the relationship between variables. For instance, since 1 is closer to 2 than to 3, you are telling your machine that `M` is "closer" to `F` (`distance = 2 - 1 = 1`) than to `other` (`distance = 3 - 1 = 2`). We want our categories to be independent. That's where one hot encoding comes into play: "[A representation of categorical variables as binary vectors](https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/)"\

Let's revert our `Gender` feature back to its original form, and see how to one-hot encode it.

In [ ]:
df_final['Gender'] = df_final['Gender'].replace(1,'M')
df_final['Gender'] = df_final['Gender'].replace(2,'F')

one_hot = pd.get_dummies(df_final['Gender'])
one_hot

As you can see, we convert 1 categorical feature into N binary features, where N = number of values that your feature can take on (2 in this example). Let's add it to our dataframe!

In [ ]:
df_final.join(one_hot)

In [ ]:
df_final.head()

Make sure you reassign the modification you made to `df_final`, or it will not save!

In [ ]:
df_final = df_final.join(one_hot)
df_final.head()

In [ ]:
df_final = df_final.drop('Gender',axis = 1)  # Drop column Gender as it is now one-hot encoded
df_final.head()

Now, several people have been wondering... What happens if you have a categorical variables with a LOT of different possible values (e.g. country data). Wouldn't one-hot encoding induce the problem of sparsity (i.e. a matrix with too many 0s and 1s)? Indeed, this could be an issue. Kate Nimegeers however had a really creative idea: one could theoretically replace the "Country" variables with 2 variables for the latitude and longitude of the centroid of each country in the dataset (or one could find a way to create some "aggregate measure" of latitude and longitude). Thus, there is lots of room for creativity :)

### 4.2 Parsing strings in your data

#### 4.2.1 Handling the `Age` feature

Recall: our `Age` feature is also non-numerical! Luckily, we have an age range, so the string can be split up into two new columns: min age and max age. Let's replace our age range in `df_final` with an average age estimate for each subject.

In [ ]:
df_final['Age'].value_counts()

For the sake of this exercise, let's replace 36+ with a range of 36-40.

In [ ]:
df_final['Age'] = df_final['Age'].replace("36+",'36-40')
df_final['Age'].value_counts()

In [ ]:
fix_age = df_final['Age'].str.split('-', 1, expand=True)
fix_age.columns = ['min','max']
fix_age
# fix_age['mean'] = (fix_age['max']+fix_age['min'])/2

In [ ]:
fix_age["min"] = fix_age['min'].astype(float)
fix_age["max"] = fix_age['max'].astype(float)
fix_age['mean'] = (fix_age['max']+fix_age['min'])/2
fix_age

In [ ]:
df_final['Age'] = fix_age['mean']
df_final

#### 4.2.2 Handling the `PSQI_BedTime` feature

Convert your bed time variable from HH:MM:SS to seconds!

In [ ]:
df_final['PSQI_BedTime']

In [ ]:
ftr = [3600,60,1]
for i in range(len(df_final['PSQI_BedTime'])):
    x = sum([a*b for a,b in zip(ftr, map(int,df_final['PSQI_BedTime'][i].split(':')))])
    df_final['PSQI_BedTime'][i] = x

In [ ]:
df_final

#### 4.2.3 Other strings that often crop up in dataframes and need to be replaced with numbers!
`
df_final = df_final.replace('FALSE',0)
df_final = df_final.replace('TRUE',1)
df_final = df_final.replace(False,0)
df_final = df_final.replace(True,1)
df_final = df_final.replace('0',0) # example of random spaces
df_final = df_final.replace(' ',np.NaN) # example of random spaces
`

#### 4.2.4 Making sure every column is of type float
The next line of code will throw an error if you forgot to replace any strings, and it will tell you what those strings are.

In [ ]:
for col in df_final.columns:
    df_final[col] = df_final[col].astype(float)

In [ ]:
df_final.head()

## 5. Handling not available (NA) and inf data:

Sometimes, Python will convert some of your values to + or - infinity, which will result in downstream errors. Convert them to NA, and then handle them as NA values.

In [ ]:
df_final = df_final.replace([np.inf, -np.inf], np.nan)

Next, you need to deal with your NA values. How many nas do you have?

In [ ]:
df_final.isna().sum()

There is a variety of ways to handle `na` data. The most simple approach is to replace `na` data with the median (or mean) value of the feature of interest. There are [other](https://towardsdatascience.com/6-different-ways-to-compensate-for-missing-values-data-imputation-with-examples-6022d9ca0779) [more](https://arxiv.org/abs/1804.11087) sophisticated data imputation techniques out there, many of which actually leverage machine learning tools (so meta)!

However, if a feature has too many NAs, you may want to remove it completely. Define a threshold for the minimal number of missing values that qualifies a feature for removal from your dataset. Here, 12 is quite stringent.

In [ ]:
threshold=12

remove_cols = []
for i in range(len(df_final.columns)):
    if (df_final.iloc[:,i].isnull().sum() >= threshold):
        remove_cols.append(df_final.columns[i])
df_final = df_final.drop(columns=remove_cols)

Next, let's replace the NA values we have left with the feature-specific median (the median is more robust against outliers than the mean is).

In [ ]:
for col in df_final.columns:
    df_final[col].fillna(df_final[col].median(), inplace=True)

In [ ]:
df_final.isna().sum().sum()
# note the difference between df_final.isna().sum() and df_final.isna().sum().sum()

## 6. Removing columns with a standard deviation of 0:

In [ ]:
df_final.std()

In [ ]:
for col in df_final.columns:
  df_final[col].fillna(df_final[col].meadian(), inplace=True)

In [ ]:
df_final = df_final.loc[:, df_final.std() > 0]

## 7. Feature scaling

You usually need to perform some sort of feature scaling to make sure that all of your variables are in the same range (this affects gradient-descent-based algorithms and distance-based algorithms==).

**Min-Max Scaling / Normalization:** X' = (X-Xmin) / (Xmax-Xmin), X' always ends up with a range of \[0,1\] \
**Standardization / Standard Scaler / Z-score):** X' = (X-mu)/sigma

Which to use? Depends on your data! ["Normalization is good to use when you know that the distribution of your data does not follow a Gaussian distribution. Standardization, on the other hand, can be helpful in cases where the data follows a Gaussian distribution."](https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/). Other [popular scaling techniques](https://www.analyticsvidhya.com/blog/2020/07/types-of-feature-transformation-and-scaling/) include the log transform (you often see this with GWAS, ie genome-wide association studies) and dividing your column-wise values by the absolute value of the maximal value of each column (max abs scaler).

Example 1: Min-Max Scaling

In [ ]:
minMaxScaled_df_final=(df_final-df_final.min())/(df_final.max()-df_final.min())

Example 2: Standard Scaling

In [ ]:
standardized_df_final=(df_final-df_final.mean())/df_final.std()

Example 3: Sklearn Min-Max Scaler\
Slightly different from the Min-Max Scaling defined above:\
`
X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
X_scaled = X_std * (max - min) + min
`

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mms=MinMaxScaler()
mms.fit(df_final)
df_final_mms=mms.transform(df_final)

# 8. Data visualization

Data visualization is a wonderful way to get to know your data in order to plan a relevant analysis or find an appropriate machine learning application. [Matplotlib](https://matplotlib.org/) and [seaborn](https://seaborn.pydata.org/) are two canonical data visualizations tools that you can use in Python.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#### 8.1 Scatter plot

In [ ]:
plt.scatter(df_final["PSQI_BedTime"],df_final["TBV"])
plt.ylabel("Total brain volume (mm3)")
plt.xlabel("Bedtime (seconds)")
plt.title("Total brain volume as a function of bed time")

#### 8.2 [Violin plot](https://chartio.com/learn/charts/violin-plot-complete-guide/)

In [ ]:
plt.violinplot(df_final['PSQI_BedTime'])
plt.ylabel("Bed time")
plt.title("Violin plot of bed times in our sample")

#### 8.3 Histogram plot

In [ ]:
sns.displot(df_final,x='TBV',kind='kde',fill=True) # smoothed histogram
plt.ylabel("Density")
plt.xlabel("Total brain volume (mm3)")
plt.title("Distribution of total brain volume in our sample")

In [ ]:
sns.histplot(df_final,x='Str_Left',fill=True,color='orange')
sns.histplot(df_final,x='Thal_Left',fill=True,color='turquoise')
plt.legend(['Left striatum','Left thalamus'])
plt.ylabel("Density")
plt.xlabel("Structure-specific volume (mm3)")
plt.title("Distribution of different structure volumes in our sample")
plt.show()

# 9. Loading local files into Google Colab

Nice tutorial [here](https://neptune.ai/blog/google-colab-dealing-with-files)

**TLDR:**

In [ ]:
from google.colab import files # files module from google.colab library

In [ ]:
uploaded = files.upload # you can then select files and upload them to your colab

# 10. Mini data visualization assignment
PS: this [list of named colours](https://matplotlib.org/3.5.0/gallery/color/named_colors.html) can help make your graphs nicer :)
<ol>
<li>Generate a scatter plot of TBV as a function of bed time, by gender.</li>
<li>Generate a violin plot of bed times in our sample for subjects over 30 years.</li>
<li>Generate two smoothed and superimposed histograms of bed times in subjects that are above and below 30 years.</li>
<li>Generate two superimposed histograms of the volume distributions in the left hemisphere (left striatum, thalamus and globus pallidus) and right hemisphere (right striatum, thalamus and globus pallidus)</li>
</ol>

# Supplementary

### Interactive plots with Plotly Python

* FYI: Plotly is based in Montreal!
* [Superb documentation](https://plotly.com/python/)
* Plotly express is more "higher-level" (tons of parameters are already set for you); can save you time but is less flexible

### Bringing it to the next level: making an App with Dash
* Documentation [here](https://dash.plotly.com/)
* html + Python